<a href="https://colab.research.google.com/github/IreneZhou0129/Google-Sheet-in-Colab/blob/main/update_text_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

import concurrent.futures
from gspread_dataframe import set_with_dataframe as dset
from google.oauth2.service_account import Credentials

In [ ]:
creds, _ = default()
gc = gspread.authorize(creds)

def read_gsheet(filename,sheet='Sheet1'):
  wb = gc.open(filename)
  sheet = wb.worksheet(sheet)

  records_data = sheet.get_all_records()
  df = pd.DataFrame.from_records(records_data)
  return df

In [1]:
def remove_articles(sentence):
    # Define the articles to be removed
    articles = {'a', 'an', 'the'}

    # Split the sentence into words, filter out the articles, and rejoin the words
    filtered_sentence = ' '.join(word for word in sentence.split() if word.lower() not in articles)

    return filtered_sentence

# Function to remove punctuation from a sentence and return the cleaned sentence
def remove_punctuation(sentence):
    return sentence.translate(str.maketrans('', '', string.punctuation))

def find_common_words_index(s1,s2):
  # Remove punctuation from s for accurate comparison
  s1_cleaned = remove_punctuation(remove_articles(s1))
  s2_cleaned = remove_punctuation(remove_articles(s2))

  # Split the cleaned s into words
  words_s1_cleaned = s1_cleaned.split()
  words_s2_cleaned = s2_cleaned.split()

  # Finding common words again after removing punctuation
  common_words_cleaned = set(words_s1_cleaned) & set(words_s2_cleaned)
  print(common_words_cleaned)

  # Initialize an empty list to store the index range for the cleaned string comparison
  index_ranges_cleaned = []

  # Find the index range of each common word in the original s1 string (to include punctuation in the index if needed)
  for word in common_words_cleaned:
      start_index = s1.find(word)
      end_index = start_index + len(word) - 1  # Adjusting for the range to be inclusive
      index_ranges_cleaned.append([start_index, end_index])
  print(index_ranges_cleaned)
  return index_ranges_cleaned

s1 = "Southern California is divided culturally, politically, and economically into distinctive regions, each containing its own culture and atmosphere, anchored usually by a city with both national and sometimes global recognition, which are often the hub of economic activity for its respective region and being home to many tourist destinations. Each region is further divided into many culturally distinct areas but as a whole combine to create the southern California atmosphere."
s2 = "What are regions anchored by that are recognized globally?"
res = find_common_words_index(s1,s2)


{'anchored', 'by', 'are', 'regions'}
[[147, 154], [164, 165], [233, 235], [90, 96]]


Docs:


*  https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets/cells#TextFormatRun
*   https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets/other#GridRange



In [ ]:
# Specify the filename of the Google Sheet and the name of the sheet within the file.
filename = 'FILENAME'
sheetName = 'SHEETNAME'

# Open the spreadsheet using the gspread library and get the specific sheet's ID.
# You can also find Sheet ID in url: https://docs.google.com/spreadsheets/d/xxxxxxxxx/edit#gid=00000000
spreadsheet = gc.open(filename)
sheetId = spreadsheet.worksheet(sheetName).id

# Read the data from the specified Google Sheet into a DataFrame and convert all data to string type.
df = read_gsheet(filename,sheet=sheetName)
df = df.astype(str)

# Iterate through each row of the DataFrame.
for i,row in df.iterrows():
  # Extract the 'question' and 'paragraph' column values for the current row.
  question = row.question
  paragraph = row.paragraph

  # Find indexes of common words between the question and the paragraph.
  find_indexes = find_common_words_index(paragraph, question) # i.e., [[90, 96], [164, 165], [147, 154], [233, 235]]

  # Define the range in the sheet where the updates will be applied.
  gridRange = {"sheetId": sheetId,
                "startRowIndex": i+1, # NOTE: Update these index numbers as needed
                "endRowIndex": i+2,
                "startColumnIndex": 6, # Column G in Google Sheet
                "endColumnIndex": 7}

  # Initial request to set the paragraph text in the specified cell.
  reqs = [
        {"updateCells": {
        "range": gridRange,
        "rows": [{"values": [{"userEnteredValue": {"stringValue": paragraph}}]}],
        "fields": "userEnteredValue"
    }}]

  # Prepare formatting requests for bold text based on found indexes.
  req_idx = []
  for idx in find_indexes:
    # Create formatting specifications for the start and end of each bold segment.
    curr_start = {"format": {"bold": True}, "startIndex": idx[0]}
    curr_end = {"format": {"bold": False}, "startIndex": idx[1]+1}
    req_idx.append(curr_start)
    req_idx.append(curr_end)

  # Add a request for applying text formatting to the list of requests.
  reqs.append({"updateCells": {
          "range": gridRange,
          "rows": [{"values": [{"textFormatRuns": req_idx
                                }]}],
          "fields": "textFormatRuns.format.bold"
      }})

  # Execute the batch update on the spreadsheet with the prepared requests.
  res = spreadsheet.batch_update({"requests": reqs})